In [2]:
from ultralytics import YOLO
import numpy as np

# Load a model
modelpose = YOLO('yolov8n-pose.pt') 

In [3]:
import os

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Thirty videos worth of data
no_sequences = 46

# Videos are going to be 30 frames in length
sequence_length = 15

In [4]:
# Define path to the main folder containing class subfolders
data_path = "C:\\Users\\ag701\\Desktop\\lstm\\LSTM\\MP_Data"  # Replace with your actual path

# Define actions based on subfolder names (modify as needed)
actions = np.array(os.listdir(data_path))  # Get list of subfolders
print(actions)


['boxing' 'jumping' 'running' 'sitting' 'squat' 'standing' 'walking']


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [6]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [7]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(15,34)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
model.load_weights('action83.h5')

In [19]:
import cv2 

colors = [(245,117,16), (117,245,16), (16,117,245), (126, 249, 255), (255, 166, 255),(16,117,245), (40, 166, 133)]
def prob_viz(res, actions, input_frame, colors, keypoints):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    top_left_x = int(keypoints[0][:, 0].min().item())  # Minimum x-coordinate across all keypoints
    top_left_y = int(keypoints[0][:, 1].min().item())-20  # Minimum y-coordinate across all keypoints
    bottom_right_x = int(keypoints[0][:, 0].max().item())  # Maximum x-coordinate across all keypoints
    bottom_right_y = int(keypoints[0][:, 1].max().item())+20 # Maximum y-coordinate across all keypoints

    # Define bounding box thickness and color
    thickness = 2
    color = (0, 255, 0)  # Green for bounding box

    # Draw the rectangle
    cv2.rectangle(output_frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)

    return output_frame



In [20]:
# # 1. New detection variables
# import cv2
# sequence = []
# sentence = []
# threshold = 0.6

# cap = cv2.VideoCapture("Dataset/jumping/6032918-hd_1920_1080_25fps.mp4")
# # Set mediapipe model 

# while cap.isOpened():

#     # Read feed
#     ret, frame = cap.read()

#     if not ret:
#         print("No frames left in video. Exiting...")
#         break
    
#     image = frame

#     # Make detections
#     results = modelpose.predict(frame)
#     # Flatten keypoints
#     # keypoints = np.array(results[0][0].keypoints.xy).flatten()
#     for r in results:
#         keypoints = np.array(r.keypoints.xy).flatten()  # Extract keypoints
        
    

#     sequence.append(keypoints)
#     sequence = sequence[-15:]
    
#     if len(sequence) == 15:
#         res = model.predict(np.expand_dims(sequence, axis=0))[0]
#         print(actions[np.argmax(res)])
        
        
#     #3. Viz logic
#         if res[np.argmax(res)] > threshold: 
#             if len(sentence) > 0: 
#                 if actions[np.argmax(res)] != sentence[-1]:
#                     sentence.append(actions[np.argmax(res)])
#             else:
#                 sentence.append(actions[np.argmax(res)])

#         if len(sentence) > 5: 
#             sentence = sentence[-5:]

        
#     cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
#     cv2.putText(image, ' '.join(sentence), (3,30), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
#     # Show to screen
#     cv2.imshow('OpenCV Feed', image)

#     # Break gracefully
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()

In [28]:
# 1. New detection variables
import cv2
import torch

sequence = []
sentence = []
threshold = 0.6

cap = cv2.VideoCapture("C:\\Users\\ag701\\Desktop\\lstm\\LSTM\\walking2.mp4")
# Set mediapipe model 

while cap.isOpened():

    # Read feed
    ret, frame = cap.read()

    if not ret:
        print("No frames left in video. Exiting...")
        break
    
    image = frame

    # Make detections
    results = modelpose.predict(frame)
    
    keyframes = []
    # # Flatten keypoints
    keypointsn = np.array(results[0][0].keypoints.xyn).flatten()
    
    if len(keypointsn) == 0:
            keypointsn = torch.zeros(1, 17, 2)
            
    for r in results:
        keypoints=[]
        keypoints = r.keypoints.xy
        print("keypoints", keypoints.shape)
        # if len(keypoints) > 0:  # Check for missing keypoints (optional)
        #     keyframes.append(keypoints)
        if len(keypoints) == 0:
            keypoints = torch.zeros(1, 17, 2)
    # if len(keyframes) > 0:  # Check for empty frames (optional)
    #     keyframes = np.array(keyframes).flatten()
    #     sequence.append(keyframes[-1])
    #     sequence = sequence[-15:]

    sequence.append(keypointsn)
    sequence = sequence[-15:]
    
    if len(sequence) == 15:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(actions[np.argmax(res)])
        
        
    #3. Viz logic
        if res[np.argmax(res)] > threshold: 
            if len(sentence) > 0: 
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5: 
            sentence = sentence[-5:]

        # Viz probabilities
        image = prob_viz(res, actions, image, colors, keypoints)
        
    cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Show to screen
    cv2.imshow('OpenCV Feed', image)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

0: 384x640 4 persons, 119.0ms
Speed: 5.0ms preprocess, 119.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 134.4ms


keypoints torch.Size([4, 17, 2])


Speed: 2.0ms preprocess, 134.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 106.4ms
Speed: 3.0ms preprocess, 106.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



keypoints torch.Size([4, 17, 2])
keypoints torch.Size([4, 17, 2])


0: 384x640 4 persons, 105.9ms
Speed: 3.0ms preprocess, 105.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 88.3ms
Speed: 2.0ms preprocess, 88.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



...

keypoints torch.Size([4, 17, 2])
1/1 [==============================] - 0s 16ms/step
boxing


0: 384x640 4 persons, 101.1ms
Speed: 0.0ms preprocess, 101.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



keypoints torch.Size([4, 17, 2])
1/1 [==============================] - 0s 14ms/step
boxing



0: 384x640 5 persons, 99.9ms
Speed: 0.0ms preprocess, 99.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


keypoints torch.Size([5, 17, 2])
1/1 [==============================] - 0s 20ms/step
boxing
